Import Required Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import geopy
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#Import Beautifulsoup
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


Get Toronto Geocode Data

In [2]:
# Use beautiful soup to parse the html content
postal_codes_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content
soup = BeautifulSoup(postal_codes_page,'html.parser')

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [4]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
df.shape

(103, 3)

In [6]:
path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
dfgc = pd.read_csv(path)
dfgc.rename(columns={'Postal Code': 'PostalCode'}, inplace=True) 
dfgc.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df1 = pd.merge(df,dfgc, on='PostalCode')
df1

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [8]:
dfT=df1[df1['Borough'].str.contains ('Toronto')].reset_index(drop=True)
dfT

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


Define a User_agent Toronto_Explorer

In [9]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_Explorerr")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto  are 43.6534817, -79.3839347.


Generate map to visualize neighborhoods

In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfT['Latitude'], dfT['Longitude'], dfT['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
dfT.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [12]:

dfT.rename(columns ={'PostalCode':'Area_ID'}, inplace=True)
dfT

,Area_ID,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


Lets do similar for the Chicago Community Area Data. We will used the 77 community areas instead of postal codes for Chicago.

In [13]:
#Import the list of Chicago Community Areas with Latitude and Longitude for each area.
dfchi = pd.read_csv('g:\coursera\data\CommAreasgeo.csv')
dfchi.head()

,Com_Area_D,Community_Area_Name,Population_2019,Area_sqm,Area_skm,density_per_sqm,Density_per_km,Coordinates,Latitude,Longitude
0,1,Rogers Park,"55,475",1.84,4.77,"30,149.46","11,640.71",42�0.6'N 87�40.2'W,42.01,-87.67
1,2,West Ridge,"78,466",3.53,9.14,"22,228.33","8,582.36",42�00'N 87�41.4'W,42.00,-87.69
2,3,Uptown,"58,979",2.32,6.01,"25,421.98","9,815.43",41�58.2'N 87�39.6'W,41.97,-87.66
3,4,Lincoln Square,"42,103",2.56,6.63,"16,446.48","6,349.99",41�58.2'N 87�41.4'W,41.97,-87.69
4,5,North Center,"35,543",2.05,5.31,"17,338.05","6,694.22",41�57'N 87�40.8'W,41.95,-87.68


In [14]:
dfchi.shape

(78, 10)

In [15]:
dfchi

,Com_Area_D,Community_Area_Name,Population_2019,Area_sqm,Area_skm,density_per_sqm,Density_per_km,Coordinates,Latitude,Longitude
0,1,Rogers Park,"55,475",1.84,4.77,"30,149.46","11,640.71",42�0.6'N 87�40.2'W,42.0100,-87.670000
1,2,West Ridge,"78,466",3.53,9.14,"22,228.33","8,582.36",42�00'N 87�41.4'W,42.0000,-87.690000
2,3,Uptown,"58,979",2.32,6.01,"25,421.98","9,815.43",41�58.2'N 87�39.6'W,41.9700,-87.660000
3,4,Lincoln Square,"42,103",2.56,6.63,"16,446.48","6,349.99",41�58.2'N 87�41.4'W,41.9700,-87.690000
4,5,North Center,"35,543",2.05,5.31,"17,338.05","6,694.22",41�57'N 87�40.8'W,41.9500,-87.680000
5,6,Lake View,"101,316",3.12,8.08,"32,473.08","12,537.86",41�56.61'N 87�39.25'W,41.9500,-87.656944
6,7,Lincoln Park,"69,465",3.16,8.18,"21,982.59","8,487.48",41�55.2'N 87�39'W,41.9200,-87.650000
7,8,Near North Side,"93,318",2.74,7.10,"34,057.66","13,149.66","41�54'00""N 87�38'04""W",41.9000,-87.634444
8,9,Edison Park,"11,112",1.13,2.93,"9,833.63","3,796.76",42�0.6'N 87�48.6'W,42.0100,-87.810000
9,10,Norwood Park,"38,084",4.37,11.32,"8,714.87","3,364.81",41�58.8'N 87�48.0'W,41.9800,-87.800000


Define a user agent Chicago_Explorer

In [16]:
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="Chicago_Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago  are 41.8755616, -87.6244212.


Generate map to visualize Community Areas

In [17]:
# create map of Chicago using latitude and longitude values
map_Chicago = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfchi['Latitude'], dfchi['Longitude'], dfchi['Community_Area_Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Chicago)  
    
map_Chicago

Combine the databases before getting Foursquare Data

In [18]:
dfchi.head()

,Com_Area_D,Community_Area_Name,Population_2019,Area_sqm,Area_skm,density_per_sqm,Density_per_km,Coordinates,Latitude,Longitude
0,1,Rogers Park,"55,475",1.84,4.77,"30,149.46","11,640.71",42�0.6'N 87�40.2'W,42.01,-87.67
1,2,West Ridge,"78,466",3.53,9.14,"22,228.33","8,582.36",42�00'N 87�41.4'W,42.00,-87.69
2,3,Uptown,"58,979",2.32,6.01,"25,421.98","9,815.43",41�58.2'N 87�39.6'W,41.97,-87.66
3,4,Lincoln Square,"42,103",2.56,6.63,"16,446.48","6,349.99",41�58.2'N 87�41.4'W,41.97,-87.69
4,5,North Center,"35,543",2.05,5.31,"17,338.05","6,694.22",41�57'N 87�40.8'W,41.95,-87.68


In [19]:
dfchi1=dfchi
dfchi1.head()

,Com_Area_D,Community_Area_Name,Population_2019,Area_sqm,Area_skm,density_per_sqm,Density_per_km,Coordinates,Latitude,Longitude
0,1,Rogers Park,"55,475",1.84,4.77,"30,149.46","11,640.71",42�0.6'N 87�40.2'W,42.01,-87.67
1,2,West Ridge,"78,466",3.53,9.14,"22,228.33","8,582.36",42�00'N 87�41.4'W,42.00,-87.69
2,3,Uptown,"58,979",2.32,6.01,"25,421.98","9,815.43",41�58.2'N 87�39.6'W,41.97,-87.66
3,4,Lincoln Square,"42,103",2.56,6.63,"16,446.48","6,349.99",41�58.2'N 87�41.4'W,41.97,-87.69
4,5,North Center,"35,543",2.05,5.31,"17,338.05","6,694.22",41�57'N 87�40.8'W,41.95,-87.68


In [20]:
dfchi1.rename(columns={'Community_Area_Name':'Area_ID'}, inplace=True)

In [21]:
dfchi1.head()

,Com_Area_D,Area_ID,Population_2019,Area_sqm,Area_skm,density_per_sqm,Density_per_km,Coordinates,Latitude,Longitude
0,1,Rogers Park,"55,475",1.84,4.77,"30,149.46","11,640.71",42�0.6'N 87�40.2'W,42.01,-87.67
1,2,West Ridge,"78,466",3.53,9.14,"22,228.33","8,582.36",42�00'N 87�41.4'W,42.00,-87.69
2,3,Uptown,"58,979",2.32,6.01,"25,421.98","9,815.43",41�58.2'N 87�39.6'W,41.97,-87.66
3,4,Lincoln Square,"42,103",2.56,6.63,"16,446.48","6,349.99",41�58.2'N 87�41.4'W,41.97,-87.69
4,5,North Center,"35,543",2.05,5.31,"17,338.05","6,694.22",41�57'N 87�40.8'W,41.95,-87.68


In [22]:
Combined_Data=dfchi.append(dfT, ignore_index=True, sort=False)

In [23]:
Combined_Data

,Com_Area_D,Area_ID,Population_2019,Area_sqm,Area_skm,density_per_sqm,Density_per_km,Coordinates,Latitude,Longitude,Borough,Neighborhood
0,1.0,Rogers Park,"55,475",1.84,4.77,"30,149.46","11,640.71",42�0.6'N 87�40.2'W,42.010000,-87.670000,NaN,NaN
1,2.0,West Ridge,"78,466",3.53,9.14,"22,228.33","8,582.36",42�00'N 87�41.4'W,42.000000,-87.690000,NaN,NaN
2,3.0,Uptown,"58,979",2.32,6.01,"25,421.98","9,815.43",41�58.2'N 87�39.6'W,41.970000,-87.660000,NaN,NaN
3,4.0,Lincoln Square,"42,103",2.56,6.63,"16,446.48","6,349.99",41�58.2'N 87�41.4'W,41.970000,-87.690000,NaN,NaN
4,5.0,North Center,"35,543",2.05,5.31,"17,338.05","6,694.22",41�57'N 87�40.8'W,41.950000,-87.680000,NaN,NaN
5,6.0,Lake View,"101,316",3.12,8.08,"32,473.08","12,537.86",41�56.61'N 87�39.25'W,41.950000,-87.656944,NaN,NaN
6,7.0,Lincoln Park,"69,465",3.16,8.18,"21,982.59","8,487.48",41�55.2'N 87�39'W,41.920000,-87.650000,NaN,NaN
7,8.0,Near North Side,"93,318",2.74,7.10,"34,057.66","13,149.66","41�54'00""N 87�38'04""W",41.900000,-87.634444,NaN,NaN
8,9.0,Edison Park,"11,112",1.13,2.93,"9,833.63","3,796.76",42�0.6'N 87�48.6'W,42.010000,-87.810000,NaN,NaN
9,10.0,Norwood Park,"38,084",4.37,11.32,"8,714.87","3,364.81",41�58.8'N 87�48.0'W,41.980000,-87.800000,NaN,NaN


In [24]:
Combined_Data_Trimmed = Combined_Data[['Area_ID','Latitude','Longitude']].copy()
Combined_Data_Trimmed

,Area_ID,Latitude,Longitude
0,Rogers Park,42.010000,-87.670000
1,West Ridge,42.000000,-87.690000
2,Uptown,41.970000,-87.660000
3,Lincoln Square,41.970000,-87.690000
4,North Center,41.950000,-87.680000
5,Lake View,41.950000,-87.656944
6,Lincoln Park,41.920000,-87.650000
7,Near North Side,41.900000,-87.634444
8,Edison Park,42.010000,-87.810000
9,Norwood Park,41.980000,-87.800000


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare Credentials and Version

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area_ID', 
                  'Area_ID Latitude', 
                  'Area_ID Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
Combined_Venues = getNearbyVenues(names=Combined_Data_Trimmed['Area_ID'],
                                   latitudes=Combined_Data_Trimmed['Latitude'],
                                   longitudes=Combined_Data_Trimmed['Longitude']
                                  )

Rogers Park
West Ridge
Uptown
Lincoln Square
North Center
Lake View
Lincoln Park
Near North Side
Edison Park
Norwood Park
Jefferson Park
Forest Glen
North Park
Albany Park
Portage Park
Irving Park
Dunning
Montclare
Belmont Cragin
Hermosa
Avondale
Logan Square
Humboldt Park
West Town
Austin
West Garfield Park
East Garfield Park
Near West Side
North Lawndale
South Lawndale
Lower West Side
Loop
Near South Side
Armour Square
Douglas
Oakland
Fuller Park
Grand Boulevard
Kenwood
Washington Park
Hyde Park
Woodlawn
South Shore
Chatham
Avalon Park
South Chicago
Burnside
Calumet Heights
Roseland
Pullman
South Deering
East Side
West Pullman
Riverdale
Hegewisch
Garfield Ridge
Archer Heights
Brighton Park
McKinley Park
Bridgeport
New City
West Elsdon
Gage Park
Clearing
West Lawn
Chicago Lawn
West Englewood
Englewood
Greater Grand Crossing
Ashburn
Auburn Gresham
Beverly
Washington Heights
Mount Greenwood
Morgan Park
O'Hare
Edgewater
Chicago
M5A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M4J
M5J
M6J
M4K
M5K
M6K


In [28]:
print(Combined_Venues.shape)
Combined_Venues.head()

(3046, 7)


,Area_ID,Area_ID Latitude,Area_ID Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rogers Park,42.01,-87.67,Morse Fresh Market,42.008087,-87.667041,Grocery Store
1,Rogers Park,42.01,-87.67,El Famous Burrito,42.010421,-87.674204,Mexican Restaurant
2,Rogers Park,42.01,-87.67,Glenwood Sunday Market,42.008525,-87.666251,Farmers Market
3,Rogers Park,42.01,-87.67,Smack Dab,42.009291,-87.666201,Bakery
4,Rogers Park,42.01,-87.67,Rogers Park Social,42.007360,-87.666265,Bar


In [29]:
Combined_Venues.groupby('Area_ID').count()

,Area_ID Latitude,Area_ID Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Area_ID,,,,,,
Albany Park,15,15,15,15,15,15
Archer Heights,8,8,8,8,8,8
Armour Square,12,12,12,12,12,12
Ashburn,8,8,8,8,8,8
Auburn Gresham,14,14,14,14,14,14
Austin,5,5,5,5,5,5
Avalon Park,3,3,3,3,3,3
Avondale,26,26,26,26,26,26
Belmont Cragin,9,9,9,9,9,9


In [30]:
print('There are {} uniques categories.'.format(len(Combined_Venues['Venue Category'].unique())))

There are 304 uniques categories.


One hot encode the Combined Venue data.

In [31]:
# one hot encoding
Combined_onehot = pd.get_dummies(Combined_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Combined_onehot['Area_ID'] = Combined_Venues['Area_ID'] 

# move neighborhood column to the first column
fixed_columns = [Combined_onehot.columns[-1]] + list(Combined_onehot.columns[:-1])
Combined_onehot = Combined_onehot[fixed_columns]

Combined_onehot.head()

,Area_ID,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,

In [32]:
Combined_onehot.shape

(3046, 305)

In [33]:
Combined_grouped = Combined_onehot.groupby('Area_ID').mean().reset_index()
Combined_grouped

,Area_ID,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,

In [34]:
Combined_grouped.shape

(115, 305)

List top 5 venues for each Area_ID

In [35]:
num_top_venues = 5

for Area in Combined_grouped['Area_ID']:
    print("----"+Area+"----")
    temp = Combined_grouped[Combined_grouped['Area_ID'] == Area].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albany Park----
                venue  freq
0                Park  0.13
1  Mexican Restaurant  0.13
2   Korean Restaurant  0.13
3              Bakery  0.07
4           BBQ Joint  0.07


----Archer Heights----
                venue  freq
0                Food  0.38
1  Mexican Restaurant  0.25
2              Bakery  0.12
3                 Bar  0.12
4  Seafood Restaurant  0.12


----Armour Square----
                venue  freq
0  Chinese Restaurant  0.25
1       Hot Dog Joint  0.08
2  Italian Restaurant  0.08
3      Sandwich Place  0.08
4    Business Service  0.08


----Ashburn----
                 venue  freq
0      Automotive Shop  0.12
1  Martial Arts School  0.12
2       Cosmetics Shop  0.12
3                  Bar  0.12
4          Snack Place  0.12


----Auburn Gresham----
                  venue  freq
0   American Restaurant  0.14
1                  Park  0.14
2  Fast Food Restaurant  0.14
3              Boutique  0.07
4           Gas Station  0.07


----Austin----
             

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Area_ID']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Combined_venues_sorted = pd.DataFrame(columns=columns)
Combined_venues_sorted['Area_ID'] = Combined_grouped['Area_ID']

for ind in np.arange(Combined_grouped.shape[0]):
    Combined_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Combined_grouped.iloc[ind, :], num_top_venues)
Combined_venues_sorted.head()

,Area_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Park,Park,Mexican Restaurant,Korean Restaurant,Bakery,BBQ Joint,Chinese Restaurant,Karaoke Bar,Taco Place,Grocery Store,Liquor Store
1,Archer Heights,Food,Mexican Restaurant,Bakery,Bar,Seafood Restaurant,Polish Restaurant,New American Restaurant,Optical Shop,Portuguese Restaurant,Opera House
2,Armour Square,Chinese Restaurant,Hot Dog Joint,Italian Restaurant,Sandwich Place,Business Service,Tanning Salon,Gas Station,Asian Restaurant,Grocery Store,Cosmetics Shop
3,Ashburn,Automotive Shop,Martial Arts School,Cosmetics Shop,Bar,Snack Place,Nightclub,Light Rail Station,Bus Station,Outdoor Sculpture,Other Great Outdoors
4,Auburn Gresham,American Restaurant,Park,Fast Food Restaurant,Boutique,Gas Station,Grocery Store,Nightclub,Pharmacy,Video Store,Locksmith


Cluster Areas

In [38]:
# set number of clusters
kclusters = 12

Combined_grouped_clustering = Combined_grouped.drop('Area_ID',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Combined_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 7, 3, 5, 0, 3, 9, 3, 3, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [39]:
# add clustering labels
Combined_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

Combined_merged =Combined_Data

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Combined_merged = Combined_merged.join(Combined_venues_sorted.set_index('Area_ID'), on='Area_ID')

Combined_merged.head() # check the last columns!

,Com_Area_D,Area_ID,Population_2019,Area_sqm,Area_skm,density_per_sqm,Density_per_km,Coordinates,Latitude,Longitude,Borough,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1.0,Rogers Park,"55,475",1.84,4.77,"30,149.46","11,640.71",42�0.6'N 87�40.2'W,42.01,-87.67,NaN,NaN,3.0,Mexican Restaurant,Pizza Place,Asian Restaurant,American Restaurant,Bar,Bakery,Theater,Train Station,Chinese Restaurant,Discount Store
1,2.0,West Ridge,"78,466",3.53,9.14,"22,228.33","8,582.36",42�00'N 87�41.4'W,42.00,-87.69,NaN,NaN,3.0,Indian Restaurant,Pakistani Restaurant,Football Stadium,Juice Bar,Fried Chicken Joint,Burmese Restaurant,Fruit & Vegetable Store,Fast Food Restaurant,Mexican Restaurant,Bar
2,3.0,Uptown,"58,979",2.32,6.01,"25,421.98","9,815.43",41�58.2'N 87�39.6'W,41.97,-87.66,NaN,NaN,5.0,Vietnamese Restaurant,Chinese Restaurant,Thai Restaurant,Coffee Shop,Music Venue,Bakery,Grocery Store,Sushi Restaurant,Pizza Place,Ethiopian Restaurant
3,4.0,Lincoln Square,"42,103",2.56,6.63,"16,446.48","6,349.99",41�58.2'N 87�41.4'W,41.97,-87.69,NaN,NaN,5.0,Café,Sandwich Place,Thai Restaurant,Bar,Gourmet Shop,Pizza Place,Eastern European Restaurant,Chinese Restaurant,Gym / Fitness Center,Gym
4,5.0,North Center,"35,543",2.05,5.31,"17,338.05","6,694.22",41�57'N 87�40.8'W,41.95,-87.68,NaN,NaN,5.0,Boutique,Pub,Bar,Arts & Crafts Store,Mobile Phone Shop,Latin American Restaurant,Breakfast Spot,Bank,German Restaurant,Beer Garden


In [40]:
Combined_merged

,Com_Area_D,Area_ID,Population_2019,Area_sqm,Area_skm,density_per_sqm,Density_per_km,Coordinates,Latitude,Longitude,Borough,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1.0,Rogers Park,"55,475",1.84,4.77,"30,149.46","11,640.71",42�0.6'N 87�40.2'W,42.010000,-87.670000,NaN,NaN,3.0,Mexican Restaurant,Pizza Place,Asian Restaurant,American Restaurant,Bar,Bakery,Theater,Train Station,Chinese Restaurant,Discount Store
1,2.0,West Ridge,"78,466",3.53,9.14,"22,228.33","8,582.36",42�00'N 87�41.4'W,42.000000,-87.690000,NaN,NaN,3.0,Indian Restaurant,Pakistani Restaurant,Football Stadium,Juice Bar,Fried Chicken Joint,Burmese Restaurant,Fruit & Vegetable Store,Fast Food Restaurant,Mexican Restaurant,Bar
2,3.0,Uptown,"58,979",2.32,6.01,"25,421.98","9,815.43",41�58.2'N 87�39.6'W,41.970000,-87.660000,NaN,NaN,5.0,Vietnamese Restaurant,Chinese Restaurant,Thai Restaurant,Coffee Shop,Music Venue,Bakery,Grocery Store,Sushi Restaurant,Pizza Place,Ethiopian Restaurant
3,4.0,Lincoln Square,"42,103",2.56,6.63,"16,446.48","6,349.99",41�58.2'N 87�41.4'W,41.970000,-87.690000,NaN,NaN,5.0,Café,Sandwich Place,Thai Restaurant,Bar,Gourmet Shop,Pizza Place,Eastern European Restaurant,Chinese Restaurant,Gym / Fitness Center,Gym
4,5.0,North Center,"35,543",2.05,5.31,"17,338.05","6,694.22",41�57'N 87�40.8'W,41.950000,-87.680000,NaN,NaN,5.0,Boutique,Pub,Bar,Arts & Crafts Store,Mobile Phone Shop,Latin American Restaurant,Breakfast Spot,Bank,German Restaurant,Beer Garden
5,6.0,Lake View,"101,316",3.12,8.08,"32,473.08","12,537.86",41�56.61'N 87�39.25'W,41.950000,-87.656944,NaN,NaN,5.0,Sports Bar,General Entertainment,Baseball Stadium,Sporting Goods Shop,Taco Place,Bus Station,Bar,BBQ Joint,Outdoor Sculpture,Mexican Restaurant
6,7.0,Lincoln Park,"69,465",3.16,8.18,"21,982.59","8,487.48",41�55.2'N 87�39'W,41.920000,-87.650000,NaN,NaN,5.0,Coffee Shop,Cosmetics Shop,Women's Store,Ice Cream Shop,Pizza Place,Gym / Fitness Center,Dessert Shop,Yoga Studio,Playground,Italian Restaurant
7,8.0,Near North Side,"93,318",2.74,7.10,"34,057.66","13,149.66","41�54'00""N 87�38'04""W",41.900000,-87.634444,NaN,NaN,5.0,Yoga Studio,Restaurant,Gym,Gym / Fitness Center,Fast Food Restaurant,Market,Massage Studio,Taco Place,Beer Bar,Paper / Office Supplies Store
8,9.0,Edison Park,"11,112",1.13,2.93,"9,833.63","3,796.76",42�0.6'N 87�48.6'W,42.010000,-87.810000,NaN,NaN,0.0,Park,Salon / Barbershop,Greek Restaurant,Sandwich Place,Thai Restaurant,Automotive Shop,Fast Food Restaurant,Pizza Place,Donut Shop,Shipping Store
9,10.0,Norwood Park,"38,084",4.37,11.32,"8,714.87","3,364.81",41�58.8'N 87�48.0'W,41.980000,-87.800000,NaN,NaN,3.0,Pizza Place,Sports Bar,Food,Moving Target,Pharmacy,Jewelry Store,Chinese Restaurant,New American Restaurant,Optical Shop,Opera House


In [41]:
Cluster_sorted=Combined_merged
Cluster_sorted

,Com_Area_D,Area_ID,Population_2019,Area_sqm,Area_skm,density_per_sqm,Density_per_km,Coordinates,Latitude,Longitude,Borough,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1.0,Rogers Park,"55,475",1.84,4.77,"30,149.46","11,640.71",42�0.6'N 87�40.2'W,42.010000,-87.670000,NaN,NaN,3.0,Mexican Restaurant,Pizza Place,Asian Restaurant,American Restaurant,Bar,Bakery,Theater,Train Station,Chinese Restaurant,Discount Store
1,2.0,West Ridge,"78,466",3.53,9.14,"22,228.33","8,582.36",42�00'N 87�41.4'W,42.000000,-87.690000,NaN,NaN,3.0,Indian Restaurant,Pakistani Restaurant,Football Stadium,Juice Bar,Fried Chicken Joint,Burmese Restaurant,Fruit & Vegetable Store,Fast Food Restaurant,Mexican Restaurant,Bar
2,3.0,Uptown,"58,979",2.32,6.01,"25,421.98","9,815.43",41�58.2'N 87�39.6'W,41.970000,-87.660000,NaN,NaN,5.0,Vietnamese Restaurant,Chinese Restaurant,Thai Restaurant,Coffee Shop,Music Venue,Bakery,Grocery Store,Sushi Restaurant,Pizza Place,Ethiopian Restaurant
3,4.0,Lincoln Square,"42,103",2.56,6.63,"16,446.48","6,349.99",41�58.2'N 87�41.4'W,41.970000,-87.690000,NaN,NaN,5.0,Café,Sandwich Place,Thai Restaurant,Bar,Gourmet Shop,Pizza Place,Eastern European Restaurant,Chinese Restaurant,Gym / Fitness Center,Gym
4,5.0,North Center,"35,543",2.05,5.31,"17,338.05","6,694.22",41�57'N 87�40.8'W,41.950000,-87.680000,NaN,NaN,5.0,Boutique,Pub,Bar,Arts & Crafts Store,Mobile Phone Shop,Latin American Restaurant,Breakfast Spot,Bank,German Restaurant,Beer Garden
5,6.0,Lake View,"101,316",3.12,8.08,"32,473.08","12,537.86",41�56.61'N 87�39.25'W,41.950000,-87.656944,NaN,NaN,5.0,Sports Bar,General Entertainment,Baseball Stadium,Sporting Goods Shop,Taco Place,Bus Station,Bar,BBQ Joint,Outdoor Sculpture,Mexican Restaurant
6,7.0,Lincoln Park,"69,465",3.16,8.18,"21,982.59","8,487.48",41�55.2'N 87�39'W,41.920000,-87.650000,NaN,NaN,5.0,Coffee Shop,Cosmetics Shop,Women's Store,Ice Cream Shop,Pizza Place,Gym / Fitness Center,Dessert Shop,Yoga Studio,Playground,Italian Restaurant
7,8.0,Near North Side,"93,318",2.74,7.10,"34,057.66","13,149.66","41�54'00""N 87�38'04""W",41.900000,-87.634444,NaN,NaN,5.0,Yoga Studio,Restaurant,Gym,Gym / Fitness Center,Fast Food Restaurant,Market,Massage Studio,Taco Place,Beer Bar,Paper / Office Supplies Store
8,9.0,Edison Park,"11,112",1.13,2.93,"9,833.63","3,796.76",42�0.6'N 87�48.6'W,42.010000,-87.810000,NaN,NaN,0.0,Park,Salon / Barbershop,Greek Restaurant,Sandwich Place,Thai Restaurant,Automotive Shop,Fast Food Restaurant,Pizza Place,Donut Shop,Shipping Store
9,10.0,Norwood Park,"38,084",4.37,11.32,"8,714.87","3,364.81",41�58.8'N 87�48.0'W,41.980000,-87.800000,NaN,NaN,3.0,Pizza Place,Sports Bar,Food,Moving Target,Pharmacy,Jewelry Store,Chinese Restaurant,New American Restaurant,Optical Shop,Opera House


In [42]:
Cluster_sorted=Combined_merged.loc[:,['Cluster Label','Area_ID','Latitude','Longitude','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue','5th Most Common Venue',
                                     '6th Most Common Venue','7th Most Common Venue','8th Most Common Venue','9th Most Common Venue','10th Most Common Venue']]
Cluster_sorted=Cluster_sorted.sort_values (by=['Cluster Label','Area_ID'])
Cluster_sorted

,Cluster Label,Area_ID,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,0.0,Auburn Gresham,41.740000,-87.660000,American Restaurant,Park,Fast Food Restaurant,Boutique,Gas Station,Grocery Store,Nightclub,Pharmacy,Video Store,Locksmith
71,0.0,Beverly,41.706000,-87.656111,Park,Cosmetics Shop,Caribbean Restaurant,Salon / Barbershop,American Restaurant,Office,Pakistani Restaurant,Outdoor Sculpture,Other Great Outdoors,Organic Grocery
43,0.0,Chatham,41.740000,-87.611667,Boutique,Park,Lounge,Discount Store,Ice Cream Shop,Donut Shop,Bar,Fast Food Restaurant,Fried Chicken Joint,Bus Station
63,0.0,Clearing,41.780000,-87.760000,Bar,Discount Store,Park,Airport,Sports Bar,Currency Exchange,Heliport,Deli / Bodega,Pizza Place,Nightclub
8,0.0,Edison Park,42.010000,-87.810000,Park,Salon / Barbershop,Greek Restaurant,Sandwich Place,Thai Restaurant,Automotive Shop,Fast Food Restaurant,Pizza Place,Donut Shop,Shipping Store
68,0.0,Greater Grand Crossing,41.760000,-87.610000,Fried Chicken Joint,Lounge,American Restaurant,Intersection,Park,Art Gallery,Currency Exchange,Donut Shop,Nightclub,New American Restaurant
96,0.0,M4N,43.728020,-79.388790,Park,Jewelry Store,Bus Line,Swim School,ATM,Noodle House,Organic Grocery,Optical Shop,Opera House,Office
14,0.0,Portage Park,41.950000,-87.760000,Sports Bar,Construction & Landscaping,Tattoo Parlor,Gas Station,Rental Car Location,Park,American Restaurant,Track,Nightclub,ATM
53,0.0,Riverdale,41.660000,-87.610000,Food,Park,Grocery Store,ATM,New American Restaurant,Optical Shop,Opera House,Office,Noodle House,Nightclub
45,0.0,South Chicago,41.740000,-87.550000,Park,Marijuana Dispensary,Dessert Shop,Pizza Place,Train Station,ATM,Nightclub,Optical Shop,Opera House,Office


In [43]:
Special_Areas=['Near North Side','Englewood','West Town','Humboldt Park','Lincoln Park','Archer Heights']
Filtered_Groups=Cluster_sorted[Cluster_sorted['Area_ID'].isin(Special_Areas)]
Filtered_Groups


,Cluster Label,Area_ID,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,3.0,Humboldt Park,41.88,-87.700000,Convenience Store,Pharmacy,Burger Joint,Bakery,Liquor Store,Pet Service,Discount Store,Seafood Restaurant,Playground,Plane
23,3.0,West Town,41.90,-87.760000,Food,Hot Dog Joint,Convenience Store,Bar,Discount Store,Nightclub,Organic Grocery,Optical Shop,Opera House,Office
67,5.0,Englewood,41.78,-87.645000,ATM,Mexican Restaurant,Sandwich Place,Grocery Store,Restaurant,Coffee Shop,Supermarket,Pharmacy,Wings Joint,Donut Shop
6,5.0,Lincoln Park,41.92,-87.650000,Coffee Shop,Cosmetics Shop,Women's Store,Ice Cream Shop,Pizza Place,Gym / Fitness Center,Dessert Shop,Yoga Studio,Playground,Italian Restaurant
7,5.0,Near North Side,41.90,-87.634444,Yoga Studio,Restaurant,Gym,Gym / Fitness Center,Fast Food Restaurant,Market,Massage Studio,Taco Place,Beer Bar,Paper / Office Supplies Store
56,7.0,Archer Heights,41.81,-87.730000,Food,Mexican Restaurant,Bakery,Bar,Seafood Restaurant,Polish Restaurant,New American Restaurant,Optical Shop,Portuguese Restaurant,Opera House


In [44]:
Toronto_clusters=pd.merge(Cluster_sorted, dfT, how='inner', on=['Area_ID'])
Toronto_clusters

,Cluster Label,Area_ID,Latitude_x,Longitude_x,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Neighborhood,Latitude_y,Longitude_y
0,0.0,M4N,43.728020,-79.388790,Park,Jewelry Store,Bus Line,Swim School,ATM,Noodle House,Organic Grocery,Optical Shop,Opera House,Office,Central Toronto,Lawrence Park,43.728020,-79.388790
1,3.0,M6H,43.669005,-79.442259,Bakery,Pharmacy,Fast Food Restaurant,Bar,Café,Park,Grocery Store,Gym / Fitness Center,Bank,Pet Store,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
2,4.0,M4J,43.685347,-79.338106,Park,Convenience Store,ATM,Nightclub,Other Great Outdoors,Organic Grocery,Optical Shop,Opera House,Office,Noodle House,East York/East Toronto,The Danforth East,43.685347,-79.338106
3,4.0,M4T,43.689574,-79.383160,Park,Restaurant,ATM,New American Restaurant,Organic Grocery,Optical Shop,Opera House,Office,Noodle House,Nightclub,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
4,4.0,M4W,43.679563,-79.377529,Park,Playground,Trail,ATM,New American Restaurant,Optical Shop,Opera House,Office,Noodle House,Nightclub,Downtown Toronto,Rosedale,43.679563,-79.377529
5,5.0,M4K,43.679557,-79.352188,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Spa,Brewery,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
6,5.0,M4L,43.668999,-79.315572,Park,Pub,Gym,Brewery,Fast Food Restaurant,Steakhouse,Burrito Place,Sushi Restaurant,Liquor Store,Fish & Chips Shop,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
7,5.0,M4M,43.659526,-79.340923,Coffee Shop,Bakery,Brewery,Café,Gastropub,American Restaurant,Yoga Studio,Diner,Bookstore,Cheese Shop,East Toronto,Studio District,43.659526,-79.340923
8,5.0,M4P,43.712751,-79.390197,Hotel,Sandwich Place,Playground,Park,Department Store,Gym / Fitness Center,Pizza Place,Gym,Breakfast Spot,Food & Drink Shop,Central Toronto,Davisville North,43.712751,-79.390197
9,5.0,M4R,43.715383,-79.405678,Clothing Store,Coffee Shop,Yoga Studio,Gym / Fitness Center,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Shoe Store,Café,Park,Central Toronto,North Toronto West,43.715383,-79.405678


In [45]:
Target_Clusters=[3.0,5.0,7.0]
Candidate_areas=Toronto_clusters[Toronto_clusters['Cluster Label'].isin(Target_Clusters)]
Candidate_areas

,Cluster Label,Area_ID,Latitude_x,Longitude_x,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Neighborhood,Latitude_y,Longitude_y
1,3.0,M6H,43.669005,-79.442259,Bakery,Pharmacy,Fast Food Restaurant,Bar,Café,Park,Grocery Store,Gym / Fitness Center,Bank,Pet Store,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
5,5.0,M4K,43.679557,-79.352188,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Spa,Brewery,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
6,5.0,M4L,43.668999,-79.315572,Park,Pub,Gym,Brewery,Fast Food Restaurant,Steakhouse,Burrito Place,Sushi Restaurant,Liquor Store,Fish & Chips Shop,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
7,5.0,M4M,43.659526,-79.340923,Coffee Shop,Bakery,Brewery,Café,Gastropub,American Restaurant,Yoga Studio,Diner,Bookstore,Cheese Shop,East Toronto,Studio District,43.659526,-79.340923
8,5.0,M4P,43.712751,-79.390197,Hotel,Sandwich Place,Playground,Park,Department Store,Gym / Fitness Center,Pizza Place,Gym,Breakfast Spot,Food & Drink Shop,Central Toronto,Davisville North,43.712751,-79.390197
9,5.0,M4R,43.715383,-79.405678,Clothing Store,Coffee Shop,Yoga Studio,Gym / Fitness Center,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Shoe Store,Café,Park,Central Toronto,North Toronto West,43.715383,-79.405678
10,5.0,M4S,43.704324,-79.388790,Dessert Shop,Sandwich Place,Italian Restaurant,Pizza Place,Gym,Café,Sushi Restaurant,Coffee Shop,Chinese Restaurant,Seafood Restaurant,Central Toronto,Davisville,43.704324,-79.388790
11,5.0,M4V,43.686412,-79.400049,Coffee Shop,Pub,Restaurant,Fried Chicken Joint,Supermarket,Sushi Restaurant,Bank,Liquor Store,Bagel Shop,Light Rail Station,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
12,5.0,M4X,43.667967,-79.367675,Coffee Shop,Pizza Place,Italian Restaurant,Café,Chinese Restaurant,Bakery,Restaurant,Pub,Pet Store,Dive Bar,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
13,5.0,M4Y,43.665860,-79.383160,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Hotel,Mediterranean Restaurant,Men's Store,Fast Food Restaurant,Pub,Downtown Toronto,Church and Wellesley,43.665860,-79.383160


A large number of Toronto Areas are in Cluster 5.  Lets cut that down to a more manageable number.

In [46]:
New_Special_Areas=['Near North Side','Englewood','Lincoln Park']
Filtered_Chicago_Groups=Combined_grouped[Combined_grouped['Area_ID'].isin(New_Special_Areas)]
Filtered_Chicago_Groups


,Area_ID,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,

In [47]:
Refined_candidate_areas=Toronto_clusters[Toronto_clusters['Cluster Label']==5.0]
Refined_candidate_areas


,Cluster Label,Area_ID,Latitude_x,Longitude_x,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Neighborhood,Latitude_y,Longitude_y
5,5.0,M4K,43.679557,-79.352188,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Spa,Brewery,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
6,5.0,M4L,43.668999,-79.315572,Park,Pub,Gym,Brewery,Fast Food Restaurant,Steakhouse,Burrito Place,Sushi Restaurant,Liquor Store,Fish & Chips Shop,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
7,5.0,M4M,43.659526,-79.340923,Coffee Shop,Bakery,Brewery,Café,Gastropub,American Restaurant,Yoga Studio,Diner,Bookstore,Cheese Shop,East Toronto,Studio District,43.659526,-79.340923
8,5.0,M4P,43.712751,-79.390197,Hotel,Sandwich Place,Playground,Park,Department Store,Gym / Fitness Center,Pizza Place,Gym,Breakfast Spot,Food & Drink Shop,Central Toronto,Davisville North,43.712751,-79.390197
9,5.0,M4R,43.715383,-79.405678,Clothing Store,Coffee Shop,Yoga Studio,Gym / Fitness Center,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Shoe Store,Café,Park,Central Toronto,North Toronto West,43.715383,-79.405678
10,5.0,M4S,43.704324,-79.388790,Dessert Shop,Sandwich Place,Italian Restaurant,Pizza Place,Gym,Café,Sushi Restaurant,Coffee Shop,Chinese Restaurant,Seafood Restaurant,Central Toronto,Davisville,43.704324,-79.388790
11,5.0,M4V,43.686412,-79.400049,Coffee Shop,Pub,Restaurant,Fried Chicken Joint,Supermarket,Sushi Restaurant,Bank,Liquor Store,Bagel Shop,Light Rail Station,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
12,5.0,M4X,43.667967,-79.367675,Coffee Shop,Pizza Place,Italian Restaurant,Café,Chinese Restaurant,Bakery,Restaurant,Pub,Pet Store,Dive Bar,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
13,5.0,M4Y,43.665860,-79.383160,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Hotel,Mediterranean Restaurant,Men's Store,Fast Food Restaurant,Pub,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
14,5.0,M5A,43.654260,-79.360636,Coffee Shop,Pub,Café,Park,Bakery,Theater,Electronics Store,Restaurant,Performing Arts Venue,Event Space,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636


In [48]:
Combined_info_refined=Refined_candidate_areas.append(Filtered_Chicago_Groups, ignore_index=True, sort=False)
Combined_info_refined

,Cluster Label,Area_ID,Latitude_x,Longitude_x,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Neighborhood,Latitude_y,Longitude_y,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,

In [49]:
Recluster_Areas=Combined_info_refined['Area_ID']
Recluster_Areas

0                 M4K
1                 M4L
2                 M4M
3                 M4P
4                 M4R
5                 M4S
6                 M4V
7                 M4X
8                 M4Y
9                 M5A
10                M5B
11                M5C
12                M5E
13                M5G
14                M5H
15                M5J
16                M5K
17                M5L
18                M5R
19                M5S
20                M5T
21                M5V
22                M5W
23                M5X
24                M6G
25                M6J
26                M6K
27                M6P
28                M6R
29                M6S
30                M7Y
31          Englewood
32       Lincoln Park
33    Near North Side
Name: Area_ID, dtype: object

In [50]:
Filtered_Combined_grouped=Combined_grouped[Combined_grouped['Area_ID'].isin(Recluster_Areas)]
Filtered_Combined_grouped

,Area_ID,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,

In [51]:
Filtered_Combined_venues_sorted=Combined_venues_sorted[Combined_venues_sorted['Area_ID'].isin(Recluster_Areas)]

In [52]:
# set number of clusters
kclusters = 6

Filtered_Combined_grouped_clustering = Filtered_Combined_grouped.drop('Area_ID',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Filtered_Combined_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 3, 5, 3, 3, 3, 3])

In [53]:
# add clustering labels
Filtered_Combined_venues_sorted=Filtered_Combined_venues_sorted.drop('Cluster Label',1)
Filtered_Combined_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

Filtered_Combined_merged =Combined_Data[Combined_Data['Area_ID'].isin(Recluster_Areas) ]

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
#Filtered_Combined_merged = Filtered_Combined_merged.join(Filtered_Combined_info_refined.set_index('Area_ID'), on='Area_ID')

Filtered_Combined_venues_sorted

,Cluster Label,Area_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,0,Englewood,ATM,Mexican Restaurant,Sandwich Place,Grocery Store,Restaurant,Coffee Shop,Supermarket,Pharmacy,Wings Joint,Donut Shop
38,1,Lincoln Park,Coffee Shop,Cosmetics Shop,Women's Store,Ice Cream Shop,Pizza Place,Gym / Fitness Center,Dessert Shop,Yoga Studio,Playground,Italian Restaurant
45,1,M4K,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Spa,Brewery
46,1,M4L,Park,Pub,Gym,Brewery,Fast Food Restaurant,Steakhouse,Burrito Place,Sushi Restaurant,Liquor Store,Fish & Chips Shop
47,3,M4M,Coffee Shop,Bakery,Brewery,Café,Gastropub,American Restaurant,Yoga Studio,Diner,Bookstore,Cheese Shop
49,5,M4P,Hotel,Sandwich Place,Playground,Park,Department Store,Gym / Fitness Center,Pizza Place,Gym,Breakfast Spot,Food & Drink Shop
50,3,M4R,Clothing Store,Coffee Shop,Yoga Studio,Gym / Fitness Center,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Shoe Store,Café,Park
51,3,M4S,Dessert Shop,Sandwich Place,Italian Restaurant,Pizza Place,Gym,Café,Sushi Restaurant,Coffee Shop,Chinese Restaurant,Seafood Restaurant
53,3,M4V,Coffee Shop,Pub,Restaurant,Fried Chicken Joint,Supermarket,Sushi Restaurant,Bank,Liquor Store,Bagel Shop,Light Rail Station
55,3,M4X,Coffee Shop,Pizza Place,Italian Restaurant,Café,Chinese Restaurant,Bakery,Restaurant,Pub,Pet Store,Dive Bar
